In [1]:
# Step 1, 2: Install Dependencies & Imports
!pip install geopandas osmnx contextily shapely rasterio scikit-learn --quiet
import os
import geopandas as gpd
import osmnx as ox
import contextily as ctx
from shapely.geometry import box
import matplotlib.pyplot as plt
from PIL import Image
import json
from tqdm import tqdm
import numpy as np
import tensorflow as tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 28.7 MB/s eta 0:00:00


In [2]:
# Step 3: Download Roads with Lanes (West Lafayette, IN)
place = "West Lafayette, Indiana, USA"
G = ox.graph_from_place(place, network_type='drive')
edges = ox.graph_to_gdfs(G, nodes=False)

# Filter valid lanes
edges = edges[edges['lanes'].astype(str).str.fullmatch(r'\d+')]
edges['lanes'] = edges['lanes'].astype(int)
edges = edges[edges['lanes'] <= 5]
edges = edges[edges['geometry'].notnull()].copy()
edges = edges.to_crs(epsg=3857)
edges['label'] = edges['lanes']
roads = edges


In [3]:
# Step 4: Download Basemap (Esri Satellite at Safe Zoom)
minx, miny, maxx, maxy = roads.total_bounds
buffer = 100
minx -= buffer; miny -= buffer; maxx += buffer; maxy += buffer

print("Downloading Esri satellite basemap...")
img, ext = ctx.bounds2img(minx, miny, maxx, maxy, zoom=16, source=ctx.providers.Esri.WorldImagery)
basemap_img = Image.fromarray(img)
basemap_img.save("basemap.png")


In [4]:
# Step 5: Crop Tiles
tile_size_px = 384
tile_meters = 48
half_m = tile_meters / 2

tiles_dir = "tiles"
os.makedirs(tiles_dir, exist_ok=True)
labels = {}

for idx, row in tqdm(roads.iterrows(), total=len(roads)):
    try:
        centroid = row.geometry.centroid
        x, y = centroid.x, centroid.y

        rel_x = (x - minx) / (maxx - minx)
        rel_y = (y - miny) / (maxy - miny)
        img_w, img_h = basemap_img.size

        px = int(rel_x * img_w)
        py = img_h - int(rel_y * img_h)

        left = px - tile_size_px // 2
        upper = py - tile_size_px // 2
        tile = basemap_img.crop((left, upper, left + tile_size_px, upper + tile_size_px))

        fname = f"tile_{idx}.png"
        tile.save(os.path.join(tiles_dir, fname))
        labels[fname] = int(row["label"])
    except Exception as e:
        print(f"Skipped {idx}: {e}")

with open("labels.json", "w") as f:
    json.dump(labels, f)

print(f"Saved {len(labels)} image tiles and labels")

100%|██████████| 766/766 [00:52<00:00, 14.61it/s]


Saved 766 image tiles and labels


In [5]:
#  Step 6: Load Dataset with Augmentation
with open("labels.json") as f:
    labels_dict = json.load(f)

filepaths = [os.path.join(tiles_dir, f) for f in labels_dict]
labels = list(labels_dict.values())

def load_img_label(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [384, 384])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    return img, label

ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
ds = ds.map(load_img_label, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.shuffle(128).batch(16).prefetch(tf.data.AUTOTUNE)

In [6]:
# Step 7: Use Pretrained ResNet50 for Lane Count Classification
base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(384, 384, 3), pooling='avg', weights='imagenet')
base_model.trainable = False  # freeze for transfer learning

x = inputs = tf.keras.Input(shape=(384, 384, 3))
x = base_model(x, training=False)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(6, activation='softmax')(x)  # 6 classes (0–5 lanes)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 384, 384, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         131,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,719,238 (90.48 MB)

 Trainable params: 131,526 (513.77 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
# Step 8: Train the Model
history = model.fit(ds, epochs=10)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 28s 245ms/step - accuracy: 0.7741 - loss: 0.9987
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 111ms/step - accuracy: 0.8264 - loss: 0.7761
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 131ms/step - accuracy: 0.8324 - loss: 0.7306
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.8447 - loss: 0.7447
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.7999 - loss: 0.7959
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 131ms/step - accuracy: 0.8311 - loss: 0.7643
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.8298 - loss: 0.7628
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - accuracy: 0.8471 - loss: 0.8514
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.8400 - loss: 0.7496
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 133ms/step - accuracy: 0.8477 - loss: 0.7662


In [10]:
# Step 9: Visualize Predictions + Confusion Matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# Preview batch predictions
for images, labels in ds.take(1):
    preds = model.predict(images)
    predicted_labels = tf.argmax(preds, axis=1).numpy()
    true_labels = labels.numpy()
    correct = predicted_labels == true_labels
    acc = np.mean(correct) * 100

    plt.figure(figsize=(16, 8))
    for i in range(min(8, len(images))):
        plt.subplot(2, 4, i+1)
        plt.imshow(images[i])
        plt.axis("off")
        plt.title(f"Pred: {predicted_labels[i]} | True: {true_labels[i]}",
                  color='green' if correct[i] else 'red')
    plt.suptitle(f"Lane Count Prediction – Accuracy: {acc:.2f}%", fontsize=16)
    plt.tight_layout()
    plt.show()

    # Confusion Matrix
    cm = tf.math.confusion_matrix(true_labels, predicted_labels, num_classes=6).numpy()
    print("Confusion Matrix:\n", cm)

Output hidden; open in https://colab.research.google.com to view.